In [61]:
import numpy as np
import pandas as pd
#import ast

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, make_scorer
from scipy.stats import pearsonr
from sklearn.ensemble import RandomForestRegressor

from tensorflow.keras.preprocessing.image import load_img
import keras.utils as image
from keras.models import Model

import keras_vggface
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace
import joblib

### <strong>Load data</strong>

In [2]:
with open('df_train.json', 'r') as f:
    json_data = f.read()

df_train = pd.read_json(json_data)

with open('df_test.json', 'r') as f:
    json_data = f.read()

df_test = pd.read_json(json_data)

In [3]:
df_train['face_encodings_vggfc6'] = df_train['face_encodings_vggfc6'].apply(lambda x: x[0])
df_test['face_encodings_vggfc6'] = df_test['face_encodings_vggfc6'].apply(lambda x: x[0])

In [4]:
df_train['face_encodings_vggresnet'] = df_train['face_encodings_vggresnet'].apply(lambda x: x[0])
df_test['face_encodings_vggresnet'] = df_test['face_encodings_vggresnet'].apply(lambda x: x[0])

In [5]:
df_train.head()

bmi  gender  is_training       name  \
0  34.207396    Male            1  img_0.bmp   
1  26.453720    Male            1  img_1.bmp   
2  34.967561  Female            1  img_2.bmp   
3  22.044766  Female            1  img_3.bmp   
4  25.845588  Female            1  img_6.bmp   

                               face_encodings_vggfc6  \
0  [-1.4681632519, -0.5275202990000001, -22.06702...   
1  [-0.2039689422, -0.7145279050000001, -62.69966...   
2  [-0.8742981553, -0.51126647, -30.3239555359, -...   
3  [-0.713303566, -0.47140464190000003, -33.48624...   
4  [0.9988322258000001, -0.32367768880000003, -21...   

                            face_encodings_vggresnet  
0  [0.0732608438, 0.1081715822, 0.7255821824, 0.0...  
1  [0.2826540768, 1.3589353561, 0.0, 0.6708666682...  
2  [0.012803124300000001, 6.6733517647, 0.0, 0.17...  
3  [0.1181243807, 2.2722120285000003, 4.283023357...  
4  [0.7783441544, 0.0, 0.25624114270000004, 0.034...

In [6]:
X_train = np.array(df_train['face_encodings_vggfc6'].tolist())
#X_train = np.array(df_train['face_encodings_vggresnet'].tolist())

y_train = np.array(df_train['bmi'])

X_test = np.array(df_test['face_encodings_vggfc6'].tolist())
#X_test = np.array(df_test['face_encodings_vggresnet'].tolist())


y_test = np.array(df_test['bmi'])

In [106]:
# model = SVR(epsilon=0.2, kernel='rbf')
# model.fit(X_train, y_train)


SVR(epsilon=0.2)

In [107]:
# # Evaluate the performance of the model
# y_pred = model.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print('Mean squared error: ', mse)

Mean squared error:  54.20578590209636


In [108]:
# corr, p_value = pearsonr(y_test, y_pred)

# print("Pearson correlation coefficient:", corr)
# print("p-value:", p_value)

Pearson correlation coefficient: 0.6443382038646485
p-value: 1.949744588887488e-89


### <strong>SVR</strong>

#### <strong>Fit on face_encodings_vggfc6</strong>

In [7]:
X_train = np.array(df_train['face_encodings_vggfc6'].tolist())
X_test = np.array(df_test['face_encodings_vggfc6'].tolist())

y_train = np.array(df_train['bmi'])
y_test = np.array(df_test['bmi'])

##### <strong><em>Grid search</em></strong>

In [110]:
%%time
param_grid = {'kernel': ['rbf', 'poly'],
    'epsilon': [0.1, 0.5, 1, 5, 10, 15, 20], 
     'C': [0.1, 1, 10, 30, 50, 100]}

# Create the SVR model
model = SVR(kernel = 'rbf')

# Create the GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=5, n_jobs=15, scoring=pearson_corr_scorer)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)


Best parameters: {'C': 10, 'epsilon': 1, 'kernel': 'rbf'}
Best score: 0.46857697198206905
CPU times: user 27.9 s, sys: 2.83 s, total: 30.7 s
Wall time: 13min 19s


##### <strong><em>Fit best model</em></strong>

In [25]:
svr_model_vgg16 = SVR(C=10, kernel='rbf', epsilon=1)
svr_model_vgg16.fit(X_train, y_train)
y_pred_vgg16 = svr_model_vgg16.predict(X_test)

mse = mean_squared_error(y_test, y_pred_vgg16)
print('Mean squared error: ', mse)

corr, p_value = pearsonr(y_test, y_pred_vgg16)
print("Pearson correlation coefficient:", corr)
print("p-value:", p_value)

Mean squared error:  50.55062485620872
Pearson correlation coefficient: 0.6507702313311919
p-value: 8.897359883790392e-92


##### <strong><em>Try on Ivy</em></strong>

In [15]:
ivy = load_img('ivy.jpg', target_size = (224, 224))
ivy = image.img_to_array(ivy)
ivy = np.expand_dims(ivy, axis = 0)
ivy = preprocess_input(ivy, version=1)

In [23]:
layer_name = 'fc6' # Specific Layer Features
vgg_model = VGGFace(model='vgg16', include_top=True, input_shape=(224, 224, 3), pooling='avg')
out = vgg_model.get_layer(layer_name).output
vggface_vggfc6 = Model(vgg_model.input, out)

In [31]:
ivy_vgg16 = vggface_vggfc6.predict(ivy) #Convolution Features
print(ivy_vgg16.shape)

1/1 [==============================] - 0s 165ms/step
(1, 4096)


In [27]:
svr_model.predict(ivy_vgg16)

array([25.33169441])

#### <strong>Fit on face_encodings_vggresnet</strong>

In [28]:
X_train_resnet = np.array(df_train['face_encodings_vggresnet'].tolist())
X_test_resnet = np.array(df_test['face_encodings_vggresnet'].tolist())

In [114]:
# model_svr_resnet = SVR(epsilon=0.2, kernel='rbf')
# model.fit(X_train_resnet, y_train)
# y_pred_resnet = model.predict(X_test_resnet)
# mse = mean_squared_error(y_test, y_pred_resnet)
# print('Mean squared error: ', mse)

Mean squared error:  54.5102882680538


In [115]:
# corr, p_value = pearsonr(y_test, y_pred_resnet)

# print("Pearson correlation coefficient:", corr)
# print("p-value:", p_value)

Pearson correlation coefficient: 0.6511451516213262
p-value: 6.472351102411023e-92


##### <strong><em>Grid search</em></strong>

In [116]:
grid_search.fit(X_train_resnet, y_train)

# Print the best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best score:", grid_search.best_score_)

Best parameters: {'C': 10, 'epsilon': 1, 'kernel': 'rbf'}
Best score: 0.4642434495116836


##### <strong><em>Fit best model</em></strong>

In [29]:
model_svr_resnet = SVR(C=10, kernel='rbf', epsilon=1)
model_svr_resnet.fit(X_train_resnet, y_train)
y_pred_resnet = model_svr_resnet.predict(X_test_resnet)

mse = mean_squared_error(y_test, y_pred_resnet)
print('Mean squared error: ', mse)

corr, p_value = pearsonr(y_test, y_pred_resnet)
print("Pearson correlation coefficient:", corr)
print("p-value:", p_value)

Mean squared error:  49.49901343777821
Pearson correlation coefficient: 0.6590397978347405
p-value: 7.157961808085102e-95


In [64]:
from sklearn.metrics import mean_absolute_error


In [65]:
mean_absolute_error(y_test, y_pred_resnet)

5.015233906474474

In [30]:
ivy = load_img('ivy.jpg', target_size = (224, 224))
ivy = image.img_to_array(ivy)
ivy = np.expand_dims(ivy, axis = 0)
ivy = preprocess_input(ivy, version=2)

In [32]:
vggface_resnet = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

In [33]:
ivy_resnet = vggface_resnet.predict(ivy) #Convolution Features
print(ivy_resnet.shape)

1/1 [==============================] - 1s 830ms/step
(1, 2048)


In [34]:
model_svr_resnet.predict(ivy_resnet)

array([23.59718345])

### <strong>Random Forest</strong>

#### <strong>Fit on face_encodings_vggfc6</strong>

#### <strong>Fit on face_encodings_vggresnet</strong>

In [40]:
X_train_resnet = np.array(df_train['face_encodings_vggresnet'].tolist())
X_test_resnet = np.array(df_test['face_encodings_vggresnet'].tolist())

In [42]:
rf_regressor = RandomForestRegressor(random_state=42)
rf_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [2, 4],  # Minimum number of samples required to be at a leaf node
}

pearson_corr_scorer = make_scorer(lambda y_true, y_pred: -pearsonr(y_true, y_pred)[0])

grid_search = GridSearchCV(estimator=rf_regressor, param_grid=rf_grid, scoring=pearson_corr_scorer, cv=5, n_jobs = 15)


In [43]:
grid_search.fit(X_train_resnet, y_train)

# Get the best parameters and the best estimator
best_params = grid_search.best_params_
best_rf_regressor = grid_search.best_estimator_

In [47]:
best_rf_regressor.fit(X_train_resnet, y_train)

ValueError: X has 4096 features, but RandomForestRegressor is expecting 2048 features as input.

In [48]:
y_pred_rf = best_rf_regressor.predict(X_test_resnet)


In [49]:
mse = mean_squared_error(y_test, y_pred_rf)
print('Mean squared error: ', mse)

corr, p_value = pearsonr(y_test, y_pred_rf)
print("Pearson correlation coefficient:", corr)
print("p-value:", p_value)

Mean squared error:  54.20988655841759
Pearson correlation coefficient: 0.6157645416200566
p-value: 1.0957892412837165e-79


In [58]:
ivy_rf = best_rf_regressor.predict(ivy_resnet) 
ivy_rf

array([25.86690652])

In [62]:
joblib.dump(model_svr_resnet, 'model_svr_resnet.h5')


['model_svr_resnet.h5']

In [ ]:
loaded_model = joblib.load('model_svr_resnet.h5')
